# **Shopee Code League 2021**
**Data Analytics - Multi-Channel Contacts**

Competition Links: https://www.kaggle.com/c/scl-2021-da

## **Problem Statements**

### **Background**

Customer service is an important element of the Shopee business, as providing a good service for our customers end-to-end is critical for business growth and brand image. Our goal is to resolve the customer’s issue within the least amount of time while requiring the least amount of customer effort.

One measure for customer effort is the number of times a customer has to approach customer service over a particular issue, this is also known as the metric “Repeat Contact Rate” or RCR. Shopee is interested in studying the RCR in order to improve the effectiveness of our customer service.

Customers can contact customer service via various channels such as the livechat function, filling up certain forms or calling in for help. Each time a customer contacts us with a new contact method, a new ticket is automatically generated. A complication arises when the same customer contacts us using different phone numbers or email addresses resulting in multiple tickets for the same issue. Hence, our challenge here is to identify how to merge relevant tickets together to create a complete picture of the customer issue and ultimately determine the RCR.

### **Task**

For each ticket, identify all contacts from each user if they have the same contact information.

For the purpose of this question, assume that all contacts from the same Phone Number / Email are the same user.

### **Basic Concepts**

*   Each Order ID represents a transaction in Shopee.
*   Each Id represents the Ticket Id made to Shopee Customer Service.
*   All Phone Numbers are stored without the country code and the country code can be ignored.
*   Contacts represent the number of times a user reached out to us in that particular ticket (Email, Call, Livechat etc.)
*   If a value is NA means that the system or agent has no record of that value.

### **Submission Format**

Determine how many contacts each user has had across the various tickets.

Two columns required:
*   ticket_id
*   ticket_trace/contact

Format: (tickets in ascending order), (total contact)
<br><br>
For each ticket, identify all other ticket_ids that belong to the same user, sorted in ascending order, as well as the total contacts the user had. If two tickets belong to the same user, the value for uid/contact for both tickets should be identical.

Your submission should have 500,000 rows (excluding headers), each with 2 columns.

## **Solutions**

### **Import Libraries**

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import time

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# path = '/content/drive/MyDrive/Colab Notebooks/Machine Learning Class/Final Projects/Shopee Code League 2021/Final Tugas Akhir/Source Code/'
path = ''

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
start_time = time.time()

### **Load Dataset File into Dataframe**

In [4]:
df = pd.read_json(path + 'contacts.json')
df

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,,1,
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,,9125983679,0,
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,,300364407,2,
...,...,...,...,...,...
499995,499995,,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv
499996,499996,,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS
499997,499997,,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku


### **Data Analysis**

In [5]:
df.shape

(500000, 5)

In [6]:
df.columns

Index(['Id', 'Email', 'Phone', 'Contacts', 'OrderId'], dtype='object')

In [7]:
total_email = df['Email'].nunique()
total_phone = df['Phone'].nunique()
total_orderid = df['OrderId'].nunique()

print("Total Unique Email: ", total_email)
print("Total Unique Phone: ", total_phone)
print("Total Unique OrderId: ", total_orderid)

Total Unique Email:  249157
Total Unique Phone:  190678
Total Unique OrderId:  189303


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Id        500000 non-null  int64 
 1   Email     500000 non-null  object
 2   Phone     500000 non-null  object
 3   Contacts  500000 non-null  int64 
 4   OrderId   500000 non-null  object
dtypes: int64(2), object(3)
memory usage: 19.1+ MB


In [9]:
df.describe()

,Id,Contacts
count,500000.000000,500000.000000
mean,249999.500000,1.996094
std,144337.711634,1.412825
min,0.000000,0.000000
25%,124999.750000,1.000000
50%,249999.500000,2.000000
75%,374999.250000,3.000000
max,499999.000000,4.000000


In [10]:
df.head()

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,,1,
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,,9125983679,0,
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,,300364407,2,


In [11]:
df.tail()

,Id,Email,Phone,Contacts,OrderId
499995,499995,,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv
499996,499996,,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS
499997,499997,,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku
499999,499999,,880053388839,0,JibSBRgzYdfzkzbTuGUXrcvDX


In [12]:
# # Example
# data = [
#       [0, 'sen@sen.com', np.NaN, np.NaN, np.NaN], 
#       [1, 'sen@sen.com', 123, np.NaN, np.NaN], 
#       [2, np.NaN, 123, np.NaN, 456], 
#       [3, np.NaN, np.NaN, np.NaN, 456]
#     ]
  
# # Create the pandas DataFrame
# df = pd.DataFrame(data, columns = ['Id', 'Email', 'Phone', 'Contacts', 'OrderId'])
# df

### **Data Pre Processing**

**Replace empty value with NaN**

In [13]:
df.isnull().sum()

Id          0
Email       0
Phone       0
Contacts    0
OrderId     0
dtype: int64

In [14]:
df = df.replace('', np.NaN)
df

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,NaN,1,NaN
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,NaN,9125983679,0,NaN
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,NaN,300364407,2,NaN
...,...,...,...,...,...
499995,499995,NaN,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv
499996,499996,NaN,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS
499997,499997,NaN,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku


**Check missing values (null)**

In [15]:
df.isnull()

,Id,Email,Phone,Contacts,OrderId
0,False,False,True,False,True
1,False,True,False,False,False
2,False,True,False,False,True
3,False,False,True,False,False
4,False,True,False,False,True
...,...,...,...,...,...
499995,False,True,False,False,False
499996,False,True,False,False,False
499997,False,True,False,False,False
499998,False,False,False,False,False


**Check sum of missing values (null)**

In [16]:
df.isnull().sum()

Id               0
Email       151394
Phone       254919
Contacts         0
OrderId     256456
dtype: int64

### **Building Solutions**

**Initialize variable dictionary (d) in order to store the tracing contact results**
<br>
**Index**: ID of each row on dataset 
<br>
**Value**: Empty set() variable

In [17]:
d_grouped_all = {i: set() for i in df.Id}
d_grouped_all

{0: set(),
 1: set(),
 2: set(),
 3: set(),
 4: set(),
 5: set(),
 6: set(),
 7: set(),
 8: set(),
 9: set(),
 10: set(),
 11: set(),
 12: set(),
 13: set(),
 14: set(),
 15: set(),
 16: set(),
 17: set(),
 18: set(),
 19: set(),
 20: set(),
 21: set(),
 22: set(),
 23: set(),
 24: set(),
 25: set(),
 26: set(),
 27: set(),
 28: set(),
 29: set(),
 30: set(),
 31: set(),
 32: set(),
 33: set(),
 34: set(),
 35: set(),
 36: set(),
 37: set(),
 38: set(),
 39: set(),
 40: set(),
 41: set(),
 42: set(),
 43: set(),
 44: set(),
 45: set(),
 46: set(),
 47: set(),
 48: set(),
 49: set(),
 50: set(),
 51: set(),
 52: set(),
 53: set(),
 54: set(),
 55: set(),
 56: set(),
 57: set(),
 58: set(),
 59: set(),
 60: set(),
 61: set(),
 62: set(),
 63: set(),
 64: set(),
 65: set(),
 66: set(),
 67: set(),
 68: set(),
 69: set(),
 70: set(),
 71: set(),
 72: set(),
 73: set(),
 74: set(),
 75: set(),
 76: set(),
 77: set(),
 78: set(),
 79: set(),
 80: set(),
 81: set(),
 82: set(),
 83: set(),
 8

**Create function in order to grouping the same data by Columns** 
<br>
**Input**: Column's name
<br>
**Output**: Dictionary of grouped data

In [18]:
def group_by_column(column_name):
    groups = df.groupby(column_name).Id.agg(lambda x: set(x))
    for ids in tqdm(groups):
        for id in ids:
            d_grouped_all[id] |= set(ids)

**Group data by Email and Update to Root Grouped Dictionary**

In [19]:
group_by_column('Email')

**Group data by Phone and Update to Root Grouped Dictionary**

In [20]:
group_by_column('Phone')

**Group data by Order Id and Update to Root Grouped Dictionary**

In [21]:
group_by_column('OrderId')

**View the result of Grouped Data**

In [22]:
d_grouped_all

{0: {0},
 1: {1, 2458, 476346},
 2: {2, 348955},
 3: {3},
 4: {4},
 5: {5, 50},
 6: {6, 38, 32871, 142067},
 7: {7},
 8: {8, 183160},
 9: {9, 13, 343161},
 10: {10, 93270},
 11: {11, 244207},
 12: {12, 480595},
 13: {9, 13, 16708, 343161, 417916},
 14: {14, 323910},
 15: {15, 17, 21197},
 16: {16},
 17: {15, 17, 21197},
 18: {18, 274269, 327057},
 19: {19, 406780},
 20: {20},
 21: {21, 499255},
 22: {22, 4562, 453177},
 23: {23, 1944, 49506},
 24: {24, 1485, 139764},
 25: {25},
 26: {26, 229592, 402598},
 27: {27, 246},
 28: {28, 1271, 26068},
 29: {29, 364569},
 30: {30, 19563},
 31: {31, 66232},
 32: {32},
 33: {33, 459, 499013},
 34: {34},
 35: {35, 51},
 36: {36},
 37: {37, 1864},
 38: {6, 38, 32871, 142067},
 39: {39, 20375, 23729, 95806, 475371},
 40: {40, 1182},
 41: {41, 168137, 275580, 337549},
 42: {42, 164913},
 43: {43, 480546},
 44: {44, 171746},
 45: {45},
 46: {46},
 47: {47},
 48: {48},
 49: {49, 50840, 179281},
 50: {5, 50, 226720, 458692},
 51: {35, 51},
 52: {52, 114

**From the result above, there is an uncomplete result data, so we need to merge the data. Here example of expected merged data:**
<br><br>
**Before:** <br>
[0] = {0,1} <br>
[1] = {1,2} <br>
[2] = {0,1,2} <br>
**After:** <br>
[0] = {0,1,2} <br>
[1] = {0,1,2} <br>
[2] = {0,1,2}

**Merge Dictionary of Grouped Data**

In [23]:
for d_index, d_values in tqdm(d_grouped_all.items()):
    for value in list(d_values):
        d_grouped_all[value] |= d_values

**View the result of merged group data**

In [24]:
d_grouped_all

{0: {0},
 1: {1, 2458, 98519, 115061, 140081, 165605, 476346},
 2: {2, 159312, 322639, 348955},
 3: {3},
 4: {4},
 5: {5, 50, 212533, 215197, 226720, 383605, 404324, 458692, 482810},
 6: {6, 38, 32871, 142067, 236367},
 7: {7},
 8: {8, 183160, 406623},
 9: {9, 13, 16708, 33415, 343161, 417916, 468927, 484896},
 10: {10, 93270},
 11: {11, 244207},
 12: {12, 160893, 480595},
 13: {9, 13, 16708, 33415, 343161, 417916, 468927, 484896},
 14: {14, 65749, 83313, 267520, 323910, 359095},
 15: {15, 17, 21197, 486106},
 16: {16},
 17: {15, 17, 21197, 486106},
 18: {18,
  10380,
  26580,
  70474,
  70621,
  79466,
  121268,
  159329,
  176080,
  235210,
  261681,
  274269,
  327057,
  382449,
  415657,
  450623},
 19: {19, 406780},
 20: {20},
 21: {21, 499255},
 22: {22,
  4562,
  106701,
  231013,
  269015,
  298102,
  408655,
  416176,
  453177,
  473446},
 23: {23, 1944, 49506, 192016},
 24: {24, 1485, 112670, 118722, 139764, 235060, 278469, 386186, 412121},
 25: {25},
 26: {26, 229592, 402598

**Create a function to Sum Contacts in a set of ID**

In [25]:
def get_sum_contact(ids_set):
    sum = 0
    for id in ids_set:
        sum += df['Contacts'].values[id]
    return sum

**Create required format data for Submission**: <br>
**tracing** = Set of tracking ticket id separated by '-' <br>
**total_contact** = The number of times a user reached out to us in that particular

In [26]:
df['d_grouped_all'] = df.Id.apply(lambda x: d_grouped_all[x])
df['tracing'] = df['d_grouped_all'].apply(lambda x: '-'.join(map(str, sorted(list(x)))))
df['total_contact'] = df['d_grouped_all'].apply(lambda x: str(get_sum_contact(x)))
df['result'] = df.tracing + ', ' + df.total_contact

**Create the Final Results for Multi Channel Contacts**

In [27]:
result = df[['Id', 'result']]
result.columns = ['ticket_id', 'ticket_trace/contact']
result

,ticket_id,ticket_trace/contact
0,0,"0, 1"
1,1,"1-2458-98519-115061-140081-165605-476346, 12"
2,2,"2-159312-322639-348955, 4"
3,3,"3, 0"
4,4,"4, 2"
...,...,...
499995,499995,"499995, 2"
499996,499996,"499996, 4"
499997,499997,"499997, 2"
499998,499998,"121111-499998, 5"


In [28]:
end_time = time.time()
print("Total Execution Time: {}".format(end_time - start_time))

Total Execution Time: 26.553970098495483


**Generate CSV file of the results data**

In [29]:
result.to_csv(path + 'results.csv', index=False)

**Accuracy = 0.95326**